### Section 1: Setup and Logging

In [ ]:
#Google API Key Setup (You can get your own API Key from Google AI Studio).
#Create a .env file on your root folder and paste your key as shown in the ".env.example" file.

import os
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

In [ ]:
import logging
import os

# Try to clean up previous logs (skip if locked)

for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        try:
            os.remove(log_file)
            print(f"🧹 Cleaned up {log_file}")
        except PermissionError:
            print(f"⚠️ Skipping {log_file} (file in use)")

# Configure logging with DEBUG log level

logging.basicConfig(
    filename = "logger.log",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)


print("✅ Logging configured")

✅ Logging configured


In [ ]:
# This is a clean-up cell, use only if necessary.
# Removes research agent folder if it already existed.
# Without this cleanup the next cell would not run if there is an existing folder.

import os
import shutil

folder_path = "C:/Google-X-Kaggle-AI-Bootcamp/Day-4/Assignments/research-agent"

if os.path.isdir(folder_path):
    print(f"Folder '{folder_path}' exists. Removing it...")
    shutil.rmtree(folder_path) # Removes the directory and its contents
    print(f"Folder '{folder_path}' removed successfully.")
else:
    print(f"Folder '{folder_path}' does not exist.")

Folder 'C:/Google-X-Kaggle-AI-Bootcamp/Day-4/Assignments/research-agent' exists. Removing it...
Folder 'C:/Google-X-Kaggle-AI-Bootcamp/Day-4/Assignments/research-agent' removed successfully.


### Section 2: Hands-On Debugging with ADK Web UI

##### Create a "Research Paper Finder" Agent

Our goal: Build a research paper finder agent that helps users find academic papers on any topic.

But first, let's intentionally create an incorrect version of the agent to practice debugging! We'll start by creating a new agent folder using the adk create CLI command.

In [4]:
# Creates a separate folder named research-agent in the root of the notebook folder.

!adk create research-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in c:\Google-X-Kaggle-AI-Bootcamp\Day-4\Assignments\research-agent:
- .env
- __init__.py
- agent.py



##### Agent definition

Next, let's create our root agent. 
- We'll configure it as an `LlmAgent`, give it a name, model and instruction.
- The `root_agent` gets the user prompt and delegates the search to the `google_search_agent`.
- Then, the agent uses the `count_papers` tool to count the number of papers returned.

**👉 Pay attention to the root agent's instructions and the `count_papers` tool parameter!**

In [5]:
%%writefile research-agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# ---- Intentionally pass incorrect datatype - `str` instead of `List[str]` ----
def count_papers(papers: List[str]):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google Search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)


# Root agent
root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.
    """,
    tools=[AgentTool(agent=google_search_agent), count_papers]
)

Overwriting research-agent/agent.py


##### Run the agent

Let's now run our agent with the `adk web --log_level DEBUG` CLI command.

**📍 The key here is `--log_level DEBUG`** - this shows us:


* **Full LLM Prompts:** The complete request sent to the language model, including system instructions, history, and tools.
* Detailed API responses from services.
* Internal state transitions and variable values.

Other log levels include: INFO, ERROR and WARNING.

In [ ]:
# You can also use -v or --verbose as a a shortcut for --log_level DEBUG.

!adk web --port 8501 --verbose c:\Google-X-Kaggle-AI-Bootcamp\Day-4\Assignments\research-agent


+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://127.0.0.1:8501.                         |
+-----------------------------------------------------------------------------+


+-----------------------------------------------------------------------------+
| ADK Web Server shutting down...                                             |
+-----------------------------------------------------------------------------+



C:\Google-X-Kaggle-AI-Bootcamp\.venv\Lib\site-packages\google\adk\cli\fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
C:\Google-X-Kaggle-AI-Bootcamp\.venv\Lib\site-packages\google\adk\auth\credential_service\in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
2025-11-30 02:53:39,279 - DEBUG - selector_events.py:64 - Using selector: SelectSelector
INFO:     Started server process [22232]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8501): only one usage of each socket

#### Test the agent in ADK web UI

##### **👉 Do: In the ADK web UI**

1. Select "research-agent" from the dropdown in the top-left.
2. In the chat interface, type: `Find latest quantum computing papers`
3. Send the message and observe the response. The agent should return a list of research papers and their count.

It looks like our agent works and we got a response! 🤔 **But wait, isn't the count of papers unusually large? Let's look at the logs and trace.** 

##### **👉 Do: Events tab - Traces in detail**

1. In the web UI, click the **"Events"** tab on the left sidebar
2. You'll see a chronological list of all agent actions
3. Click on any event to expand its details in the bottom panel
4. Try clicking the **"Trace"** button to see timing information for each step.
5. **Click the `execute_tool count_papers` span. You'll see that the function call to `count_papers` returns the large number as the response**.
6. Let's look at what was passed as input to this function. 
7. **Find the `call_llm` span corresponding to the `count_papers` function call**.

##### **👉 Do: Inspect the Function call in Events:**

- Click on the specific span to open the Events tab.
- Examine the `function_call`, focusing on the `papers` argument.
- Notice that `root_agent` passes the list of `papers` as a **str** instead of a **List[str]** - there's our bug! 

**Other Observability questions you can now answer from logs and adk web:**
- **Efficiency**: Is the agent making optimal tool choices?
- **Reasoning Quality**: Are the prompts well-structured and context-appropriate?
- **Performance**: Look at the traces to identify which steps take the longest?
- **Failure Diagnosis**: When something goes wrong, where exactly did it fail?

**Key Learning:** Core debugging pattern: `symptom → logs → root cause → fix`.

**Debugging Victory:** You just went from "Agent mysteriously failed" to "I know exactly why and how to fix it!" This is the power of observability!


---
### 🧑‍💻 Section 3: Logging in production

**🎯 Great! You can now debug agent failures using ADK web UI and DEBUG logs.**

But what happens when you move beyond development? Real-world scenarios where you need to move beyond the web UI:

**❌ Problem 1: Production Deployment**
```
You: "Let me open the ADK web UI to check why the agent failed"
DevOps: "Um... this is a production server. No web UI access."
You: 😱 "How do I debug production issues?"
```

**❌ Problem 2: Automated Systems** 
```
You: "The agent runs 1000 times per day in our pipeline"
Boss: "Which runs are slow? What's our success rate?"
You: 😰 "I'd have to manually check the web UI 1000 times..."
```

**💡 The Solution:**

We need a way to capture observability data or in other words, **add logs to our code**. 

👉 In traditional software development, this is done by adding log statements in Python functions - **and agents are no different!** We need to add log statements to our agent and a common approach is to add log statements to **Plugins**.


##### How to add logs for production observability?

A Plugin is a custom code module that runs automatically at various stages of your agent's lifecycle. Plugins are composed of "**Callbacks**" which provide the hooks to interrupt an agent's flow. Think of it like this:

- **Your agent workflow**: User message → Agent thinks → Calls tools → Returns response
- **Plugin hooks into this**: Before agent starts → After tool runs → When LLM responds → etc.
- **Plugin contains your custom code**: Logging, monitoring, security checks, caching, etc.

![image.png](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/plugins-callbacks.png)

##### Callbacks

Callbacks are the **atomic components inside a Plugin** - these are just Python functions that run at specific points in an agent's lifecycle! **Callbacks are grouped together to create a Plugin.**

There are different kinds of callbacks such as:
* **before/after_agent_callbacks** - runs before/after an agent is invoked
* **before/after_tool_callbacks** - runs before/after a tool is called
* **before/after_model_callbacks** - similarly, runs before/after the LLM model is called
* **on_model_error_callback** - which runs when a model error is encountered

![image.png](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/types_of_callbacks.png)

In [7]:
print("----- EXAMPLE PLUGIN - DOES NOTHING ----- ")

import logging
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin


# Applies to all agent and model calls
class CountInvocationPlugin(BasePlugin):
    """A custom plugin that counts agent and tool invocations."""

    def __init__(self) -> None:
        """Initialize the plugin with counters."""
        super().__init__(name="count_invocation")
        self.agent_count: int = 0
        self.tool_count: int = 0
        self.llm_request_count: int = 0

    # Callback 1: Runs before an agent is called. You can add any custom logic here.
    async def before_agent_callback(
        self, *, agent: BaseAgent, callback_context: CallbackContext
    ) -> None:
        """Count agent runs."""
        self.agent_count += 1
        logging.info(f"[Plugin] Agent run count: {self.agent_count}")

    # Callback 2: Runs before a model is called. You can add any custom logic here.
    async def before_model_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest
    ) -> None:
        """Count LLM requests."""
        self.llm_request_count += 1
        logging.info(f"[Plugin] LLM request count: {self.llm_request_count}")

----- EXAMPLE PLUGIN - DOES NOTHING ----- 


![image.png](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/count-invocation-plugin.png)

#### ADK's built-in `LoggingPlugin`

But you don't have to define all the callbacks and plugins to capture *standard* Observability data in ADK. Instead, ADK provides a built-in **LoggingPlugin** that automatically captures all agent activity:

- 🚀 User messages and agent responses
- ⏱️ Timing data for performance analysis
- 🧠 LLM requests and responses for debugging
- 🔧 Tool calls and results
- ✅ Complete execution traces

In [8]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


def count_papers(papers: List[str]):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="Use the google_search tool to find information on the given topic. Return the raw search results.",
    tools=[google_search],
)

# Root agent
research_agent_with_plugin = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 
   
   You must follow these steps:
   1) Find research papers on the user provided topic using the 'google_search_agent'. 
   2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
   3) Return both the list of research papers and the total number of papers.
   """,
    tools=[AgentTool(agent=google_search_agent), count_papers],
)

print("✅ Agent created")

✅ Agent created


In [9]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=research_agent_with_plugin,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)

print("✅ Runner configured")

✅ Runner configured


In [10]:
print("🚀 Running agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await runner.run_debug("Find recent papers on quantum computing")

🚀 Running agent with LoggingPlugin...
📊 Watch the comprehensive logging output below:


 ### Created new session: debug_session_id

User > Find recent papers on quantum computing
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-eea07e2c-42bf-4268-965e-9b7e9579582a
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: research_paper_finder_agent
[logging_plugin]    User Content: text: 'Find recent papers on quantum computing'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-eea07e2c-42bf-4268-965e-9b7e9579582a
[logging_plugin]    Starting Agent: research_paper_finder_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: research_paper_finder_agent
[logging_plugin]    Invocation ID: e-eea07e2c-42bf-4268-965e-9b7e9579582a
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[l

---

### Summary

**❓ When to use which type of Logging?**
1. **Development debugging?** → Use `adk web --log_level DEBUG`
2. **Common production observability?** → Use `LoggingPlugin()` 
3. **Custom requirements?** → Build Custom Callbacks and Plugins
